In [9]:
import pandas as pd
import numpy as np
import pickle
import random

In [10]:
train = pd.read_csv('../input2/train.csv')
train2 = pd.read_csv('../input2/dev1.csv')
train3 = pd.read_csv('../input2/dev2.csv')
train2 = train3.append(train2).drop_duplicates('id',keep='first')
test = pd.read_csv('../input2/test1.csv')
test2 = pd.read_csv('../input2/test2.csv')
train.shape,train2.shape,test.shape

((8500, 8), (640, 8), (1000, 8))

In [11]:
same_id_dict = {}
for line in open('./same_id.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    same_id_dict[int(array[1])] = int(array[0])

for id,v in same_id_dict.items():
    temp_id = id
    while(True):
        if same_id_dict.get(temp_id, -1) != -1:
            temp_id = same_id_dict[temp_id]
        else:
            break
    same_id_dict[id] = temp_id

id_label = {}
cache = train[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_label[cache[i,0]] = cache[i,2]


id_label2 = {}
cache = train2[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_label[cache[i,0]] = cache[i,2]

ids_list = list(id_label.keys()) 

cache = test2[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_label2[cache[i,0]] = random.choice([0,1])
ids_list2 = list(id_label2.keys()) 

userset = {}
img_pairs = []
text_pairs = []
for line in open('./img_pairs.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    img_pairs.append([int(array[0]),int(array[1])])
    if int(array[0]) not in test2.id.values or int(array[1]) not in test2.id.values:
        continue
    userset[int(array[0])] = userset.get(int(array[0]), {})
    userset[int(array[0])][0] = userset[int(array[0])].get(0, 0) + 1
    userset[int(array[1])] = userset.get(int(array[1]), {})
    userset[int(array[1])][0] = userset[int(array[1])].get(0, 0) + 1
    
for line in open('./text_pairs.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    text_pairs.append([int(array[0]),int(array[1])])
    if int(array[0]) not in test2.id.values or int(array[1]) not in test2.id.values:
        continue
    userset[int(array[0])] = userset.get(int(array[0]), {})
    userset[int(array[0])][1] = userset[int(array[0])].get(1, 0) + 1
    userset[int(array[1])] = userset.get(int(array[1]), {})
    userset[int(array[1])][1] = userset[int(array[1])].get(1, 0) + 1

    
id_leak = {}
for line in open('./img_pairs.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    if int(array[0]) not in test2.id.values or int(array[1]) not in test2.id.values:
        continue
    id = int(array[0])
    id2 = int(array[1])
    if userset[id].get(0,0) > 0 and userset[id].get(1,0) > 0:
        pass
    elif userset[id].get(0,0) > 0 or userset[id].get(1,0) > 0:
        if userset[id2].get(0,0) > 0 and userset[id2].get(1,0) > 0:
            id_leak[id] = 0
    id = int(array[1])
    id2 = int(array[0])
    if userset[id].get(0,0) > 0 and userset[id].get(1,0) > 0:
        pass
    elif userset[id].get(0,0) > 0 or userset[id].get(1,0) > 0:
        if userset[id2].get(0,0) > 0 and userset[id2].get(1,0) > 0:
            id_leak[id] = 0
    
for line in open('./text_pairs.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    if int(array[0]) not in test2.id.values or int(array[1]) not in test2.id.values:
        continue
    id = int(array[0])
    id2 = int(array[1])
    if userset[id].get(0,0) > 0 and userset[id].get(1,0) > 0:
        pass
    elif userset[id].get(0,0) > 0 or userset[id].get(1,0) > 0:
        if userset[id2].get(0,0) > 0 and userset[id2].get(1,0) > 0:
            id_leak[id] = 0
    id = int(array[1])
    id2 = int(array[0])
    if userset[id].get(0,0) > 0 and userset[id].get(1,0) > 0:
        pass
    elif userset[id].get(0,0) > 0 or userset[id].get(1,0) > 0:
        if userset[id2].get(0,0) > 0 and userset[id2].get(1,0) > 0:
            id_leak[id] = 0    
ids_list3 = [x for x in ids_list2]
print(len(img_pairs),len(text_pairs),len(ids_list3))

6937 6814 2000


In [12]:
id_pair = []
for pairs in [text_pairs,img_pairs]:
    cluster = {}
    id2cluster = {}
    cluster2id = {}
    clusterid = 0 
    count1 = 0
    count2 = 0
    for line in pairs:
        id0 = line[0]
        id1 = line[1]
        if id_label.get(id0,-1) == -1 and id0 not in test2.id.values:
            continue
        if id_label.get(id1,-1) == -1 and id1 not in test2.id.values:
            continue
        if id0 in test2.id.values or id1 in test2.id.values:
            if same_id_dict.get(id0,id0) != same_id_dict.get(id1,id1):
                id_pair.append(line)
    
len(id_pair)

2650

In [20]:
f = open("id_bias.pkl", 'rb')
id_bias,id_bias_test1 = pickle.loads(f.read())
f.close()
len(id_bias_test1),len(id_bias)

(218, 195)

In [21]:

id_pred3_all = []
count = 0

for file in ['submission7.pkl','submission12.pkl','submission8.pkl']:
    f = open(file, 'rb')
    id_pred_2,id_pred2_2,id_mmf = pickle.loads(f.read())
    f.close()
    id_pred3 = {}
    for id in ids_list3:
        if id_pred2_2.get(id,-1) != -1:
            id_pred3[id] =  (id_pred_2[id] * 0.3 + np.mean(id_pred2_2[id]) * 0.3 + id_mmf[id] * 0.4)
        else:
            id_pred3[id] =  id_pred_2[id] * 0.6 + id_mmf[id] * 0.4

        if id_bias.get(id,-1) != -1:
            id_pred3[id] = id_bias[id] * 0.8 + id_pred3[id] * 0.2
        elif id_bias_test1.get(same_id_dict.get(id,id),-1) != -1:
            id_pred3[id] = id_bias_test1[same_id_dict.get(id,id)] * 0.8 + id_pred3[id] * 0.2  


    for id in id_pair:
        a0 = 0.7
        a1 = 0.7
        if id[0] in test2.id.values and id[1] in test2.id.values:
            
            b = abs(np.mean(id_pred2_2[id[0]]) - np.mean(id_pred2_2[id[1]])) / 0.4
            b = max(1.0,min(2.0,b))
            a0 = a0/b
            a1 = a1/b
            
            if np.mean(id_pred2_2[id[0]]) > np.mean(id_pred2_2[id[1]]):
                id_pred3[id[0]] = id_pred3[id[0]] * a0 + 1.0 * (1 - a0)
                id_pred3[id[1]] = id_pred3[id[1]] * a1 + 0.0 * (1 - a1)
            else:
                id_pred3[id[0]] = id_pred3[id[0]] * a0 + 0.0 * (1 - a0)
                id_pred3[id[1]] = id_pred3[id[1]] * a1 + 1.0 * (1 - a1)
            
        elif id_label.get(id[0],-1) != -1 and id_rule.get(id[1],-1) == -1 and id_rule_test1.get(same_id_dict.get(id[1],id[1]),-1) == -1:
            a0 = 0.85
            a1 = 0.85
            if id_label[id[0]] > np.mean(id_pred2_2[id[1]]):
                id_pred3[id[1]] = id_pred3[id[1]] * a1 + 0.0 * (1 - a1)
            else:
                id_pred3[id[1]] = id_pred3[id[1]] * a1 + 1.0 * (1 - a1)
        elif id_label.get(id[1],-1) != -1 and id_bias.get(id[0],-1) == -1 and id_bias_test1.get(same_id_dict.get(id[0],id[0]),-1) == -1:
            a0 = 0.85
            a1 = 0.85
            if id_label[id[1]] > np.mean(id_pred2_2[id[0]]):
                id_pred3[id[0]] = id_pred3[id[0]] * a1 + 0.0 * (1 - a1)
            else:
                id_pred3[id[0]] = id_pred3[id[0]] * a1 + 1.0 * (1 - a1)
    
    for id in id_pair:
        a0 = 0.3
        a1 = 0.3
        if id[0] in test2.id.values and id[1] in test2.id.values:
            
            if id_mmf[id[0]] > id_mmf[id[1]]:
                if 1 - id_pred3[id[0]] > id_pred3[id[1]]:
                    id_pred3[id[0]] = np.clip(id_pred3[id[0]] + a0,0,1)
                else:
                    id_pred3[id[1]] = np.clip(id_pred3[id[1]] - a1,0,1)

            else:
                if 1 - id_pred3[id[1]] > id_pred3[id[0]]:
                    id_pred3[id[1]] = np.clip(id_pred3[id[1]] + a0,0,1)
                else:
                    id_pred3[id[0]] = np.clip(id_pred3[id[0]] - a1,0,1)

    for id in id_pair:
        a0 = 0.6
        a1 = 0.6
        if id[0] in test2.id.values and id[1] in test2.id.values:
            
            meanscore = (id_pred3[id[0]] + id_pred3[id[1]]) * 0.5
            id_pred3[id[0]] = id_pred3[id[0]] - (meanscore - 0.5) * a0
            id_pred3[id[1]] = id_pred3[id[1]] - (meanscore - 0.5) * a1

    for id in test2.id.values:
        if id_bias.get(id,-1) != -1:
            id_pred3[id] = id_bias[id] * 0.5 + id_pred3[id] * 0.5
        elif id_bias_test1.get(same_id_dict.get(id,id),-1) != -1:
            id_pred3[id] = id_bias_test1[same_id_dict.get(id,id)] * 0.5 + id_pred3[id] * 0.5
    for id in test2.id.values:
        if userset.get(id,-1) != -1:
            if userset[id].get(0,0) > 0 and userset[id].get(1,0) > 0:
                id_pred3[id] =  1 * 0.5 + id_pred3[id] * 0.5
        if id_leak.get(id,-1) != -1:
            id_pred3[id] =  0 * 0.4 + id_pred3[id] * 0.6
            
    id_pred3_all.append(id_pred3)
    
f = open('./merge2_final4.csv','w')
print("id,proba,label",file = f )        
for id in test2.id.values:
    id_pred3[id] = np.mean([x[id] for x in id_pred3_all])
    print(",".join([str(id),str(round(id_pred3[id],4)),str(1 if id_pred3[id] > 0.5 else 0)]),file = f )
f.close()    
